In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Crear carpeta para guardar los resultados
if not os.path.exists('apartado6'):
    os.makedirs('apartado6')

In [4]:
# Leer el archivo `clientes_segmentados.csv`
file_path = 'datos_telecom_preprocesado.csv'
df_segmentados = pd.read_csv(file_path)

In [6]:
# Seleccionar columnas numéricas
numerical_cols = df_segmentados.select_dtypes(include=['number']).columns
data_normalizada = df_segmentados[numerical_cols]#.drop(columns=['Cluster'])  # Excluir la columna de cluster
labels = df_segmentados['Cluster']

KeyError: 'Cluster'

In [ ]:
# Coeficiente de Silueta Promedio
silhouette_avg = silhouette_score(data_normalizada, labels)
print(f"\nCoeficiente de Silueta Promedio: {silhouette_avg:.4f}")

In [ ]:
# Distancia Intra-cluster (cohesión)
clusters = df_segmentados['Cluster'].unique()
intra_cluster_distances = {}
for cluster in clusters:
    cluster_data = data_normalizada[labels == cluster]
    centroid = cluster_data.mean(axis=0)
    distances = np.linalg.norm(cluster_data - centroid, axis=1)
    intra_cluster_distances[cluster] = distances.mean()
print("\nDistancia promedio Intra-cluster (cohesión):")
for cluster, distance in intra_cluster_distances.items():
    print(f"Cluster {cluster}: {distance:.4f}")

In [ ]:












# Distancia Inter-cluster (separación)
centroids = df_segmentados.groupby('Cluster').mean(numeric_only=True)  # Promedio solo numéricos
distances = euclidean_distances(centroids)
np.fill_diagonal(distances, np.nan)  # Ignorar distancias intra-cluster
inter_cluster_distances = np.nanmean(distances)
print(f"\nDistancia promedio Inter-cluster (separación): {inter_cluster_distances:.4f}")

# Guardar resultados en un archivo TXT
validation_path = 'apartado6/validation_results.txt'
with open(validation_path, 'w') as file:
    file.write(f"Coeficiente de Silueta Promedio: {silhouette_avg:.4f}\n")
    file.write("\nDistancia promedio Intra-cluster (cohesión):\n")
    for cluster, distance in intra_cluster_distances.items():
        file.write(f"Cluster {cluster}: {distance:.4f}\n")
    file.write(f"\nDistancia promedio Inter-cluster (separación): {inter_cluster_distances:.4f}\n")
    file.write("\nEtiquetas sugeridas para cada cluster:\n")
    file.write("Cluster 0: Clientes jóvenes de ingresos bajos con alto nivel de gasto.\n")
    file.write("Cluster 1: Clientes mayores con ingresos altos y alto gasto mensual.\n")
    file.write("Cluster 2: Clientes jóvenes de ingresos bajos y bajo gasto mensual.\n")
    file.write("Cluster 3: Clientes jóvenes con ingresos bajos y gasto promedio.\n")

print(f"\nResultados de validación guardados en '{validation_path}'.")

# Visualización adicional: Heatmap de distancias inter-cluster
plt.figure(figsize=(8, 6))
sns.heatmap(distances, annot=True, fmt=".2f", cmap='coolwarm', xticklabels=clusters, yticklabels=clusters)
plt.title('Matriz de Distancias Inter-cluster')
plt.xlabel('Cluster')
plt.ylabel('Cluster')
heatmap_path = 'apartado6/heatmap_inter_cluster.png'
plt.savefig(heatmap_path)
plt.show()

# Visualización de PCA (Variación explicada)
pca = PCA()
pca.fit(data_normalizada)
explained_variance = pca.explained_variance_ratio_

plt.figure(figsize=(8, 6))
plt.bar(range(1, len(explained_variance)+1), explained_variance, alpha=0.7, align='center')
plt.step(range(1, len(explained_variance)+1), np.cumsum(explained_variance), where='mid', label='Cumulative Variance')
plt.xlabel('Número de Componentes Principales')
plt.ylabel('Proporción de Variación Explicada')
plt.title('PCA - Variación Explicada')
plt.legend(loc='best')
pca_path = 'apartado6/pca_variation.png'
plt.savefig(pca_path)
plt.show()

# Histogramas de distancias intra e inter-cluster
plt.figure(figsize=(8, 6))
sns.histplot(list(intra_cluster_distances.values()), kde=True, bins=10, color='blue', label='Intra-cluster')
plt.axvline(np.mean(list(intra_cluster_distances.values())), color='blue', linestyle='--', label='Media Intra-cluster')
plt.title('Distribución de Distancias Intra-cluster')
plt.legend()
plt.savefig('apartado6/histograma_intra_cluster.png')
plt.show()

plt.figure(figsize=(8, 6))
sns.histplot(distances[~np.isnan(distances)].flatten(), kde=True, bins=10, color='red', label='Inter-cluster')
plt.axvline(inter_cluster_distances, color='red', linestyle='--', label='Media Inter-cluster')
plt.title('Distribución de Distancias Inter-cluster')
plt.legend()
plt.savefig('apartado6/histograma_inter_cluster.png')
plt.show()

# Conclusión en consola
print("\nVisualizaciones y métricas adicionales generadas y guardadas en 'apartado6'.")
